In [ ]:
pip install evalml

In [ ]:
import evalml
import numpy as np
import pandas as pd
df = pd.read_csv("../input/analytics-vidhya-job-a-thon-may-2021/train_s3TEQDk.csv")
test = pd.read_csv('../input/analytics-vidhya-job-a-thon-may-2021/test_mSzZ8RL.csv')
X = df[df.columns[:-1]].set_index('ID')
y = df.Is_Lead.astype('int64')
X_train, X_test, y_train, y_test = evalml.preprocessing.split_data(X, y, problem_type='binary')

In [ ]:
X_train,y_train

In [ ]:
from evalml.automl import AutoMLSearch
automl = AutoMLSearch(X_train=X_train, y_train=y_train, problem_type='binary',max_batches=3)
automl.search()

In [ ]:
from evalml.data_checks import TargetLeakageDataCheck

target_leakage_check = TargetLeakageDataCheck(pct_corr_threshold=0.8)
results = target_leakage_check.validate(X, y)

for message in results['warnings']:
    print("Warning:", message['message'])

for message in results['errors']:
    print("Error:", message['message'])


In [ ]:
from evalml.data_checks import *


target_leakage_check = TargetLeakageDataCheck(pct_corr_threshold=0.8)
results = target_leakage_check.validate(X, y)

for message in results['warnings']:
    print("Warning:", message['message'])

for message in results['errors']:
    print("Error:", message['message'])



In [ ]:
from evalml.data_checks import OutliersDataCheck

outliers_check = OutliersDataCheck()
results = outliers_check.validate(X, y)

for message in results['warnings']:
    print("Warning:", message['message'])

for message in results['errors']:
    print("Error:", message['message'])

In [ ]:
test.isna().sum()

In [ ]:
pip install mlbox

In [ ]:
from mlbox.preprocessing import *
from mlbox.optimisation import *
from mlbox.prediction import *

In [ ]:
paths = ["../input/analytics-vidhya-job-a-thon-may-2021/train_s3TEQDk.csv","../input/analytics-vidhya-job-a-thon-may-2021/test_mSzZ8RL.csv"]
target_name = "Is_Lead"

In [ ]:
rd = Reader(sep = ",")
df = rd.train_test_split(paths, target_name) 

In [ ]:
dft = Drift_thresholder()
df = dft.fit_transform(df) 

In [ ]:
opt = Optimiser(scoring = "roc_auc", n_folds=5)

In [ ]:
space = {
    'ne__categorical_strategy' : {"space" : ['<NULL>', 'most_frequent']},

        'ce__strategy' : {"space" : ["random_projection", "entity_embedding"]},
    
        'est__strategy':{"search":"choice",
                                  "space":["LightGBM"]},    
        'est__n_estimators':{"search":"choice",
                                  "space":[150]},    
        'est__colsample_bytree':{"search":"uniform",
                                  "space":[0.8,0.95]},
        'est__subsample':{"search":"uniform",
                                  "space":[0.8,0.95]},
        'est__max_depth':{"search":"choice",
                                  "space":[5,6,7,8,9]},
        'est__learning_rate':{"search":"choice",
                                  "space":[0.07]}, 
    'est__boosting_type' : { "space" : ['gbdt', 'rf', 'goss', 'dart']}
    
        }

params = opt.optimise(space, df,15)

In [ ]:
prd = Predictor()
prd.fit_predict(params, df)

In [ ]:
test = pd.read_csv('../input/analytics-vidhya-job-a-thon-may-2021/test_mSzZ8RL.csv')

In [ ]:
res = pd.read_csv('./save/Is_Lead_predictions.csv')['1.0']

In [ ]:
final = pd.DataFrame()
final['ID'] = test['ID']
final['Is_Lead'] = res

In [ ]:
final.to_csv('mlbox_final.csv',index=False)